**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import os
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization
from keras.initializers import RandomNormal

if not os.path.isdir("random"):
    os.makedirs("random")
if not os.path.isdir("FCN"):
    os.makedirs("FCN")
if not os.path.isdir("CNN"):
    os.makedirs("CNN")

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

With the function act, the actor chooses a random action with probability $\epsilon$ and act via its policy with a probability $1-\epsilon$. Therefore, ```epsilon``` defines the "proportion" of exploration of the actor. It's a way of improving policies by ensuring exploration in the environment to discover some state-action pairs that can never be encountred under a deterministic policy for example. It is then essential to find an optimal trade-off between exploitation and exploration by determining an optimal value of epsilon. In general, we favourise exploration over exploitation in the begining of the learning and decay the exploration gradually. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        
        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        # coordinate of the rat
        self.x = 0
        self.y = 1
        
        # self time
        self.t = 0
        
        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=15 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array ```board``` defines the distribution of the rewards over the $N\times N$ grid. It stores the reward of being in each cell that we can access through the coordinates x and y of the agent. If one reward is collected by the agent during an episode, this reward is reset to 0.

The array ```position``` defines the borders (forbidden states) of the grid, the allowed states and the position of the agent. At each step of the game, a value of -1 is assigned for the cells representing the borders (ie the states that can't be visited), a value of 0 is assigned for all the other cells that can be visited, and a value of 1 is assigned for the current cell occupied by the agent.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e+1))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})".format(win, lose, score/(1+e)))
        
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()
test(agent,env,epochs_test,prefix='random/random')
HTML(display_videos('random/random1.mp4'))

Win/lose count 12.0/18.0. Average score (-6.0)
Win/lose count 9.0/9.0. Average score (-3.0)
Win/lose count 12.0/15.0. Average score (-3.0)
Win/lose count 6.5/16.0. Average score (-4.625)
Win/lose count 10.0/19.0. Average score (-5.5)
Win/lose count 13.5/13.0. Average score (-4.5)
Win/lose count 9.5/13.0. Average score (-4.357142857142857)
Win/lose count 17.5/11.0. Average score (-3.0)
Win/lose count 9.5/16.0. Average score (-3.388888888888889)
Win/lose count 10.5/16.0. Average score (-3.6)
Win/lose count 14.0/14.0. Average score (-3.272727272727273)
Win/lose count 12.0/12.0. Average score (-3.0)
Win/lose count 10.0/16.0. Average score (-3.230769230769231)
Win/lose count 11.5/14.0. Average score (-3.1785714285714284)
Win/lose count 8.5/18.0. Average score (-3.6)
Final score: -3.6


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__1)__
$$\boxed{
\begin{align*} 
Q^\pi(s,a)&=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=E_{p^{\pi}}[r(s_0,a_0) + \gamma\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]\\
          &=r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a]\\
          &=r(s,a) + E_{(s',a')\sim p(.|s,a)}\left(\gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          &=E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']\right)\\
          &=E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{\pi}(s',a') \right)
\end{align*}
}$$

$$\boxed{
\begin{align*}
\pi^*(s') &= arg\max_{a'} Q^{*}(s',a')\\
Q^*(s,a)&= E_{(s',a')\sim p(.|s,a)}\left(r(s,a) + \gamma Q^{*}(s',a') \right)\\
        &= E_{s'\sim p(.|s,a)\\ a'= \pi^*(.|s')}\left(r(s,a) + \gamma Q^{*}(s',a') \right)\\
        &= E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*}(s',a') \right)\\
\end{align*}
}$$

__3)__ We've shown that $Q^*(s,a)=E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q^{*}(s',a') \right)$, therefore, $Q^*$ is the solution of the equation $F(Q) = 0$ with $$F(Q) = E_{s' \sim \pi^*(.|s,a)}\left(r(s,a) + \gamma \max_{a'}Q(s',a') - Q(s,a)\right)$$

Minimizing $\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$ will hence lead us to find the optimal Q.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)>=self.max_memory:
            self.memory.pop(0)
        
        self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e+1) % 10 == 0:
            env.draw(prefix+str(e+1))

        # Update stats
        score += win-lose
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None]),1)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            # Saving current state
            input_states[i] = s_
            
            # Get current Q(s,.)
            target_q[i] = self.model.predict(s_[None])
            
            # Update Q(s,a)
            if game_over_:
                # There is no next state, we take only the reward
                target_q[i,a_] = r_
            else:
                # We update target_q by computing the TD error 
                target_q[i,a_] = r_ + self.discount*np.max(self.model.predict(n_s_[None]),axis=1)
                
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        # Flattening the input frame
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        # First fully connected layer
        model.add(Dense(20, activation='relu'))#, kernel_initializer=RandomNormal(stddev=0.02)))
        # Final fully connected layer
        model.add(Dense(4))

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
size = 13
T=200
temperature=0.3
epochs_train=50
epochs_test=15

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='FCN/fc_train')
HTML(display_videos('FCN/fc_train50.mp4'))

Epoch 001/050 | Loss 0.0033 | Win/lose count 0.5/2.0 (-1.5)
Epoch 002/050 | Loss 0.0157 | Win/lose count 3.0/5.0 (-2.0)
Epoch 003/050 | Loss 0.0040 | Win/lose count 2.5/2.0 (0.5)
Epoch 004/050 | Loss 0.0131 | Win/lose count 3.5/5.0 (-1.5)
Epoch 005/050 | Loss 0.0438 | Win/lose count 4.0/1.0 (3.0)
Epoch 006/050 | Loss 0.0134 | Win/lose count 3.0/5.0 (-2.0)
Epoch 007/050 | Loss 0.0059 | Win/lose count 3.5/1.0 (2.5)
Epoch 008/050 | Loss 0.0129 | Win/lose count 1.0/2.0 (-1.0)
Epoch 009/050 | Loss 0.0059 | Win/lose count 1.5/2.0 (-0.5)
Epoch 010/050 | Loss 0.0070 | Win/lose count 4.0/5.0 (-1.0)
Epoch 011/050 | Loss 0.0077 | Win/lose count 1.5/2.0 (-0.5)
Epoch 012/050 | Loss 0.0066 | Win/lose count 2.0/5.0 (-3.0)
Epoch 013/050 | Loss 0.0043 | Win/lose count 3.0/6.0 (-3.0)
Epoch 014/050 | Loss 0.0089 | Win/lose count 5.0/5.0 (0.0)
Epoch 015/050 | Loss 0.0022 | Win/lose count 3.0/5.0 (-2.0)
Epoch 016/050 | Loss 0.0164 | Win/lose count 4.5/4.0 (0.5)
Epoch 017/050 | Loss 0.0039 | Win/lose count 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        
        # First convolutional layer with kernel size 3, stride 1 and 16 filters
        model.add(Conv2D(16, (3,3), input_shape=(5,5,self.n_state), activation="relu"))
        
        # Second convolutional layer with kernel size 3, stride 1 and 32 filters
        model.add(Conv2D(32, (3,3), activation="relu"))
        
        # Flattening and feeding the output to a 4 units fully connected layer
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='CNN/cnn_train')
HTML(display_videos('CNN/cnn_train50.mp4'))

Epoch 001/050 | Loss 0.0077 | Win/lose count 3.0/5.0 (-2.0)
Epoch 002/050 | Loss 0.0029 | Win/lose count 2.0/5.0 (-3.0)
Epoch 003/050 | Loss 0.0023 | Win/lose count 3.0/8.0 (-5.0)
Epoch 004/050 | Loss 0.0033 | Win/lose count 5.0/2.0 (3.0)
Epoch 005/050 | Loss 0.0116 | Win/lose count 4.5/2.0 (2.5)
Epoch 006/050 | Loss 0.0105 | Win/lose count 4.0/3.0 (1.0)
Epoch 007/050 | Loss 0.0064 | Win/lose count 5.0/1.0 (4.0)
Epoch 008/050 | Loss 0.0070 | Win/lose count 2.0/3.0 (-1.0)
Epoch 009/050 | Loss 0.0070 | Win/lose count 2.0/0 (2.0)
Epoch 010/050 | Loss 0.0208 | Win/lose count 7.0/3.0 (4.0)
Epoch 011/050 | Loss 0.0043 | Win/lose count 4.5/4.0 (0.5)
Epoch 012/050 | Loss 0.0022 | Win/lose count 8.0/3.0 (5.0)
Epoch 013/050 | Loss 0.0673 | Win/lose count 6.5/4.0 (2.5)
Epoch 014/050 | Loss 0.0028 | Win/lose count 5.0/2.0 (3.0)
Epoch 015/050 | Loss 0.0030 | Win/lose count 3.5/1.0 (2.5)
Epoch 016/050 | Loss 0.0012 | Win/lose count 6.5/2.0 (4.5)
Epoch 017/050 | Loss 0.0546 | Win/lose count 2.5/3.0 (

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='CNN/cnn_trainmodel.h5', name_model='CNN/cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='FCN/fc_trainmodel.h5', name_model='FCN/fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='CNN/cnn_test')

print('\nTest of the FC')
test(agent_fc,env,epochs_test,prefix='FCN/fc_test')

Test of the CNN
Win/lose count 10.0/1.0. Average score (9.0)
Win/lose count 11.5/4.0. Average score (8.25)
Win/lose count 10.5/2.0. Average score (8.333333333333334)
Win/lose count 14.5/1.0. Average score (9.625)
Win/lose count 13.5/2.0. Average score (10.0)
Win/lose count 8.5/0. Average score (9.75)
Win/lose count 13.0/1.0. Average score (10.071428571428571)
Win/lose count 13.0/4.0. Average score (9.9375)
Win/lose count 6.5/1.0. Average score (9.444444444444445)
Win/lose count 9.5/4.0. Average score (9.05)
Win/lose count 11.0/1.0. Average score (9.136363636363637)
Win/lose count 11.5/3.0. Average score (9.083333333333334)
Win/lose count 12.5/2.0. Average score (9.192307692307692)
Win/lose count 3.5/1.0. Average score (8.714285714285714)
Win/lose count 3.5/1.0. Average score (8.3)
Final score: 8.3

Test of the FC
Win/lose count 7.5/2.0. Average score (5.5)
Win/lose count 6.5/3.0. Average score (4.5)
Win/lose count 3.5/7.0. Average score (1.8333333333333333)
Win/lose count 2.5/8.0. Aver

In [16]:
HTML(display_videos('CNN/cnn_test14.mp4'))

In [17]:
HTML(display_videos('FCN/fc_test15.mp4'))

__Comments__

> We notice that the agent doesn't explore the map well and sometimes get stuck in a position, when there is no positive reward within his visibility, especially for the agent with the fully connected network.

> When we increase the temperature, positive rewards cells get numerous, which helps the agent explore the map since more positive cells gets in his visibility, and therefore the final average score gets higher.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    # Starting epsilon
    epsilon_start = agent.epsilon
    
    # Minimum value that epsilon could get
    epsilon_stop= 0.1

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        # Exponential decay for epsilon-greedy exploration
        agent.set_epsilon((epsilon_start-epsilon_stop)*np.exp(-e/8) + epsilon_stop)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if (e+1) % 10 == 0:
            env.draw(prefix+str(e+1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.2f}/{:.2f} ({:.2f}) | Epsilon : {:.2f}"
              .format(e, epoch, loss, win, lose, win-lose, agent.epsilon))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        
        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        
        # coordinate of the cat
        self.x = 0
        self.y = 1
        
        # self time
        self.t = 0
        
        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
            

        self.malus_position[self.x, self.y] = 0.1 
        self.t = self.t + 1
        
        reward = self.board[self.x, self.y] - int(train)*self.malus_position[self.x, self.y]
        self.board[self.x, self.y] = 0

        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)  
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        self.malus_position[self.x, self.y] = 0.1
        
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2) 

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [19]:
# Parameters
size = 13
T=200
temperature=0.3
epochs_train=100
epochs_test=15

# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.7, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='CNN/cnn_train_explore')
HTML(display_videos('CNN/cnn_train_explore100.mp4'))

Epoch 000/100 | Loss 0.0190 | Win/lose count 13.20/34.80 (-21.60) | Epsilon : 0.70
Epoch 001/100 | Loss 0.0133 | Win/lose count 8.80/24.90 (-16.10) | Epsilon : 0.63
Epoch 002/100 | Loss 0.0104 | Win/lose count 10.40/30.50 (-20.10) | Epsilon : 0.57
Epoch 003/100 | Loss 0.0152 | Win/lose count 9.60/21.70 (-12.10) | Epsilon : 0.51
Epoch 004/100 | Loss 0.0052 | Win/lose count 8.40/24.00 (-15.60) | Epsilon : 0.46
Epoch 005/100 | Loss 0.0083 | Win/lose count 12.40/25.00 (-12.60) | Epsilon : 0.42
Epoch 006/100 | Loss 0.0072 | Win/lose count 9.60/22.70 (-13.10) | Epsilon : 0.38
Epoch 007/100 | Loss 0.0088 | Win/lose count 6.40/26.50 (-20.10) | Epsilon : 0.35
Epoch 008/100 | Loss 0.0075 | Win/lose count 12.40/24.00 (-11.60) | Epsilon : 0.32
Epoch 009/100 | Loss 0.0077 | Win/lose count 10.00/19.60 (-9.60) | Epsilon : 0.29
Epoch 010/100 | Loss 0.0077 | Win/lose count 5.20/20.80 (-15.60) | Epsilon : 0.27
Epoch 011/100 | Loss 0.0092 | Win/lose count 6.40/24.50 (-18.10) | Epsilon : 0.25
Epoch 012/10

In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='CNN/cnn_test_explore')
HTML(display_videos('CNN/cnn_test_explore15.mp4'))

Win/lose count 19.0/3.0. Average score (16.0)
Win/lose count 13.5/3.0. Average score (13.25)
Win/lose count 13.5/2.0. Average score (12.666666666666666)
Win/lose count 23.0/5.0. Average score (14.0)
Win/lose count 15.5/1.0. Average score (14.1)
Win/lose count 14.0/4.0. Average score (13.416666666666666)
Win/lose count 13.5/5.0. Average score (12.714285714285714)
Win/lose count 17.5/3.0. Average score (12.9375)
Win/lose count 19.0/4.0. Average score (13.166666666666666)
Win/lose count 19.0/8.0. Average score (12.95)
Win/lose count 19.0/7.0. Average score (12.863636363636363)
Win/lose count 26.0/4.0. Average score (13.625)
Win/lose count 16.0/1.0. Average score (13.73076923076923)
Win/lose count 15.0/3.0. Average score (13.607142857142858)
Win/lose count 24.0/2.0. Average score (14.166666666666666)
Final score: 14.166666666666666


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***